#Ingest data from landing to bronze using Autoloader

We access to the storage account and set the paths to the files

In [0]:
###Se establece la "account key" para acceso al Azure Data Lake Storage Gen2
spark.conf.set("fs.azure.account.key.tfmgendos.dfs.core.windows.net","QT/ULR9KopMmUZmz55avGjsJf/cuoAdFd/HUT3qDxs5pLllRhvXwyQEsqn8b1RKn91hv7lTuhb9K+AStoUFp+A==")

basepath="abfss://tfm@tfmgendos.dfs.core.windows.net/"

dbutils.fs.ls(basepath)


Out[3]: [FileInfo(path='abfss://tfm@tfmgendos.dfs.core.windows.net/bronze/', name='bronze/', size=0, modificationTime=1662129140000),
 FileInfo(path='abfss://tfm@tfmgendos.dfs.core.windows.net/gold/', name='gold/', size=0, modificationTime=1662129150000),
 FileInfo(path='abfss://tfm@tfmgendos.dfs.core.windows.net/landing/', name='landing/', size=0, modificationTime=1662129135000),
 FileInfo(path='abfss://tfm@tfmgendos.dfs.core.windows.net/silver/', name='silver/', size=0, modificationTime=1662129145000)]

In [0]:

# autoloader table and checkpoint paths
#bronze path
WEODatabaseByCountry_Bronce = basepath + "bronze/fmi/"
bronzeCheckpoint = basepath + "bronze/fmi/checkpoint/"
bronzeSchema = basepath + "bronze/fmi/schema/"

#landing path
landingZoneLocation = basepath + "landing/fmi/"



In [0]:
dbutils.fs.ls(landingZoneLocation)

Out[6]: [FileInfo(path='abfss://tfm@tfmgendos.dfs.core.windows.net/landing/fmi/11013b45-67af-45b2-a836-af92cf343940.csv', name='11013b45-67af-45b2-a836-af92cf343940.csv', size=10770241, modificationTime=1662576543000),
 FileInfo(path='abfss://tfm@tfmgendos.dfs.core.windows.net/landing/fmi/e69ec695-4626-4b64-b5e9-09b54968a5f6.csv', name='e69ec695-4626-4b64-b5e9-09b54968a5f6.csv', size=10770241, modificationTime=1662477618000),
 FileInfo(path='abfss://tfm@tfmgendos.dfs.core.windows.net/landing/fmi/weobycountrygroup/', name='weobycountrygroup/', size=0, modificationTime=1662567262000)]

In [0]:
# "cloudFiles" indicates the use of Auto Loader
# We are using csv and give a schema location so that the current schema can be saved and infered easily
dfBronze = spark.readStream.format("cloudFiles") \
  .option("cloudFiles.format", "csv") \
  .option("cloudFiles.schemaLocation", bronzeSchema) \
  .option("header", "true") \
  .option("delimiter", "\t") \
  .load(landingZoneLocation)




In [0]:
from pyspark.sql import functions as F

#Replace the space character in column names to "_"
dfBronze = dfBronze.select([F.col(col).alias(col.replace(' ', '_')) for col in dfBronze.columns])

In [0]:
# The stream will shut itself off when it is finished based on the trigger once feature
# The checkpoint location saves the state of the ingest when it is shut off so we know where to pick up next time
dfBronze.writeStream \
  .format("delta") \
  .trigger(once=True) \
  .option("checkpointLocation", bronzeCheckpoint) \
  .option("header", "true")\
  .start(WEODatabaseByCountry_Bronce)

Out[9]: <pyspark.sql.streaming.StreamingQuery at 0x7f8252061bb0>

In [0]:
#Load and verify bronze table
dfBronze = spark.read.format("delta").load(WEODatabaseByCountry_Bronce)

dfBronze.show()



+----------------+---+----------------+-----------+--------------------+--------------------+--------------------+--------+-----------------------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----+----+----+----+----+----+----+---------------------+--------------------+
WEO_Country_Code|ISO|WEO_Subject_Code| Country| Subject_Descriptor| Subject_Notes| Units| Scale|Country/Series-specific_Notes|1980|1981|1982|1983|1984|1985|1986|1987|1988|1989|1990|1991|1992|1993|1994|1995|1996|1997|1998|1999|2000|2001| 2002| 2003| 2004| 2005| 2006| 2007| 2008| 2009| 2010| 2011| 2012| 2013| 2014| 2015| 2016| 2017| 2018| 2019| 2020|2021|2022|2023|2024|2025|2026|2027|Estimates_Start_After| _rescued_data|
+----------------+---+----------------+-----------+--------------------+--------------------+--------------------+--------+-----------------------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----+----+----+----+----+----+----+---------------------+--------------------+
 512|AFG| NGDP_R|Afghanistan|Gross domestic pr...|Expressed in bill...| National currency|Billions| Source: National ...| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| 453.484| 492.903| 496.209| 554.910| 584.658| 662.650| 688.247| 829.924| 899.956| 958.266| 1,092.118| 1,154.178| 1,185.306| 1,197.012| 1,222.917| 1,255.288| 1,270.216| 1,319.902| 1,288.869| n/a| n/a| n/a| n/a| n/a| n/a| n/a| 2020|{"_c58":"","_file...|
 512|AFG| NGDP_RPCH|Afghanistan|Gross domestic pr...|Annual percentage...| Percent change| null| See notes for: G...| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| 8.692| 0.671| 11.830| 5.361| 13.340| 3.863| 20.585| 8.438| 6.479| 13.968| 5.683| 2.697| 0.988| 2.164| 2.647| 1.189| 3.912| -2.351| n/a| n/a| n/a| n/a| n/a| n/a| n/a| 2020|{"_c58":"","_file...|
 512|AFG| NGDP|Afghanistan|Gross domestic pr...|Expressed in bill...| National currency|Billions| Source: National ...| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| 178.756| 220.013| 246.210| 304.926| 345.817| 427.495| 517.509| 607.227| 711.759| 836.222| 1,033.591| 1,116.827| 1,183.039| 1,226.570| 1,222.917| 1,285.460| 1,327.690| 1,469.596| 1,547.289| n/a| n/a| n/a| n/a| n/a| n/a| n/a| 2020|{"_c58":"","_file...|
 512|AFG| NGDPD|Afghanistan|Gross domestic pr...|Values are based ...| U.S. dollars|Billions| See notes for: G...| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| 4.367| 4.553| 5.146| 6.167| 6.925| 8.556| 10.297| 12.066| 15.325| 17.890| 20.293| 20.170| 20.616| 20.057| 18.020| 18.883| 18.401| 18.876| 20.136| n/a| n/a| n/a| n/a| n/a| n/a| n/a| 2020|{"_c58":"","_file...|
 512|AFG| PPPGDP|Afghanistan|Gross domestic pr...|These data form t...|Purchasing power ...|Billions| See notes for: G...| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| 19.661| 21.792| 22.527| 25.982| 28.220| 32.848| 34.772| 42.198| 46.309| 50.334| 59.945| 63.784| 69.444| 72.056| 70.098| 74.712| 77.406| 81.873| 80.912| n/a| n/a| n/a| n/a| n/a| n/a| n/a| 2020|{"_c58":"","_file...|
 512|AFG| NGDP_D|Afghanistan|Gross domestic pr...|The GDP deflator ...| Index| null| See notes for: G...| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a| n/a|

In [0]:
dfBronze.count()


Out[68]: 8625